In [22]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types



In [12]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [7]:

df = spark.read \
    .option("header","true") \
    .csv('../week01/data/taxi+_zone_lookup.csv')

df.head()

Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR')

In [ ]:
!head taxi+_zone_lookup.csv.csv 



In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet

--2023-04-16 10:51:11--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2123:2e00:b:20a5:b140:21, 2600:9000:2123:3000:b:20a5:b140:21, 2600:9000:2123:f000:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2123:2e00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘fhv_tripdata_2021-01.parquet’

fhv_tripdata_2021-0 100%[===================>]  11,33M  2,86MB/s    in 17s     

2023-04-16 10:51:29 (681 KB/s) - ‘fhv_tripdata_2021-01.parquet’ saved [11886281/11886281]



In [2]:
!wc -1 fhv_tripdata_2021-01.parquet

wc: invalid option -- '1'
Try 'wc --help' for more information.


In [4]:
df = spark.read \
    .option("header","true") \
    .parquet('../data/fhvhv_tripdata_2021-01.parquet')

df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [16]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [8]:
import pyarrow as pa
import pyarrow.dataset as ds

In [6]:
df_pandas = df.limit(100).toPandas()

In [7]:
df_pandas.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02682,B02682,2020-12-31 21:28:09,2020-12-31 21:31:42,2020-12-31 21:33:44,2020-12-31 21:49:07,230,166,5.26,...,1.98,2.75,NaN,0.00,14.99,N,N,,N,N
1,HV0003,B02682,B02682,2020-12-31 21:45:56,2020-12-31 21:55:19,2020-12-31 21:55:19,2020-12-31 22:18:21,152,167,3.65,...,1.63,0.00,NaN,0.00,17.06,N,N,,N,N
2,HV0003,B02764,B02764,2020-12-31 21:21:15,2020-12-31 21:22:41,2020-12-31 21:23:56,2020-12-31 21:38:05,233,142,3.51,...,1.25,2.75,NaN,0.94,12.98,N,N,,N,N
3,HV0003,B02764,B02764,2020-12-31 21:39:12,2020-12-31 21:42:37,2020-12-31 21:42:51,2020-12-31 21:45:50,142,143,0.74,...,0.70,2.75,NaN,0.00,7.41,N,N,,N,N
4,HV0003,B02764,B02764,2020-12-31 21:46:11,2020-12-31 21:47:17,2020-12-31 21:48:14,2020-12-31 22:08:42,143,78,9.20,...,2.41,2.75,NaN,0.00,22.44,N,N,,N,N


In [8]:
df_pandas.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [34]:
df = spark.read \
    .option("header","true") \
    .parquet('../data/fhv_tripdata_2021-01.parquet')

df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2020-12-31 22:27:00|2020-12-31 22:44:00|        null|        null|   null|                B00009|
|              B00009|2020-12-31 22:50:00|2020-12-31 23:07:00|        null|        null|   null|                B00009|
|              B00013|2020-12-31 22:01:00|2020-12-31 23:51:00|        null|        null|   null|                B00013|
|              B00037|2020-12-31 22:13:09|2020-12-31 22:21:26|        null|        72.0|   null|                B00037|
|              B00037|2020-12-31 22:38:31|2020-12-31 22:53:44|        null|        61.0|   null|                B00037|
|              B00037|2020-12-31 22:59:0

In [35]:
df.write.option("header","true") \
.csv('../data/fhv_tripdata_2021-01.csv')

In [18]:
df_pandas = df.limit(100).toPandas()

In [20]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   dispatching_base_num    100 non-null    object        
 1   pickup_datetime         100 non-null    datetime64[ns]
 2   dropOff_datetime        100 non-null    datetime64[ns]
 3   PUlocationID            9 non-null      float64       
 4   DOlocationID            84 non-null     float64       
 5   SR_Flag                 0 non-null      float64       
 6   Affiliated_base_number  100 non-null    object        
dtypes: datetime64[ns](2), float64(3), object(2)
memory usage: 5.6+ KB


In [36]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])



In [40]:
df_csv =spark.read\
    .option('header','true') \
    .schema(schema) \
    .csv('../data/fhv_tripdata_2021-01.csv')

In [39]:
df_csv.head()

23/04/16 16:09:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag, Affiliated_base_number
 Schema: hvfhs_license_num, dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, SR_Flag
Expected: hvfhs_license_num but found: dispatching_base_num
CSV file: file:///home/leticia/Documents/data_engineering_zoocamp/week04/data/fhv_tripdata_2021-01.csv/part-00001-2fcd3c34-b85b-4e54-acac-a0754b31acfb-c000.csv


Row(hvfhs_license_num='B00009', dispatching_base_num='2020-12-31T22:27:00.000-02:00', pickup_datetime=datetime.datetime(2020, 12, 31, 21, 44), dropoff_datetime=None, PULocationID=None, DOLocationID=None, SR_Flag='B00009')

In [41]:
df_csv = df_csv.repartition(24)

In [42]:
df_csv.write.parquet('../data/fhvhv/2021/01')

23/04/16 16:10:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag, Affiliated_base_number
 Schema: hvfhs_license_num, dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, SR_Flag
Expected: hvfhs_license_num but found: dispatching_base_num
CSV file: file:///home/leticia/Documents/data_engineering_zoocamp/week04/data/fhv_tripdata_2021-01.csv/part-00001-2fcd3c34-b85b-4e54-acac-a0754b31acfb-c000.csv


23/04/16 16:10:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag, Affiliated_base_number
 Schema: hvfhs_license_num, dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, SR_Flag
Expected: hvfhs_license_num but found: dispatching_base_num
CSV file: file:///home/leticia/Documents/data_engineering_zoocamp/week04/data/fhv_tripdata_2021-01.csv/part-00000-2fcd3c34-b85b-4e54-acac-a0754b31acfb-c000.csv


23/04/16 16:10:36 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


23/04/16 16:10:38 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


23/04/16 16:10:39 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [43]:
df_csv = spark.read.parquet('../data/fhvhv/2021/01')

In [46]:
df_csv.show()


+-----------------+--------------------+-------------------+----------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+----------------+------------+------------+-------+
|           B00350|2021-01-01T14:22:...|2021-01-01 14:37:43|            null|        null|        null| B03085|
|           B01536|2021-01-02T06:06:...|2021-01-02 06:18:33|            null|        null|        null| B01536|
|           B03028|2021-01-04T13:27:...|2021-01-04 13:32:51|            null|        null|        null| B03028|
|           B03268|2021-01-03T07:42:...|2021-01-03 08:36:14|            null|        null|        null| B03268|
|           B01312|2021-01-03T03:32:...|2021-01-03 03:34:12|            null|        null|        null| B02867|
|           B01282|2021-01-05T09:12:...|2021-01-05 09:20:08|            null|        null|        null| 

In [44]:
df_csv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [47]:
df_csv.select('pickup_datetime','dropoff_datetime').filter(df_csv.hvfhs_license_num =='B00350').show()

+-------------------+----------------+
|    pickup_datetime|dropoff_datetime|
+-------------------+----------------+
|2021-01-01 14:37:43|            null|
|2021-01-05 14:49:01|            null|
|2021-01-01 15:09:03|            null|
|2021-01-02 05:30:37|            null|
|2021-01-05 15:00:58|            null|
|2021-01-02 17:47:17|            null|
|2021-01-01 15:28:29|            null|
|2021-01-01 05:29:46|            null|
|2021-01-03 10:31:56|            null|
|2021-01-02 11:06:30|            null|
|2021-01-05 05:28:14|            null|
|2021-01-01 08:36:18|            null|
|2021-01-02 06:29:35|            null|
|2021-01-03 03:25:25|            null|
|2021-01-04 07:45:01|            null|
|2021-01-08 07:43:03|            null|
|2021-01-06 14:21:19|            null|
|2021-01-06 07:51:54|            null|
|2021-01-08 09:56:56|            null|
|2021-01-09 19:40:31|            null|
+-------------------+----------------+
only showing top 20 rows

